In [ ]:
from kafka import KafkaProducer
import tweepy
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from tweepy import Stream
import json

myTopic = 'partita'
lista=[]

# Declare a Python class (MyListener) that inherit from tweepy.StreamListener
# Then we override the on_status method that will be automatically used once
# Twitter "sends" us a new tweet.
class MyListener(StreamListener):
    
    def __init__(self, producer, producer_topic):
        super().__init__()
        self.producer = producer
        self.producer_topic = producer_topic
     

    def on_status(self, status):
        
        
        
        is_retweet = hasattr(status, "retweeted_status")

        if hasattr(status, 'retweeted_status'):
            try:
                text = status.retweeted_status.extended_tweet["full_text"]
            except:
                text = status.retweeted_status.text
        else:
            try:
                text = status.extended_tweet["full_text"]
            except AttributeError:
                text = status.text
        msg= {
                  "user_id": status.user.id,
                  "date": status.created_at,
                  "username": status.user.name,
                  "text": text,
                  "location": status.user.location,
                'is_RT' : is_retweet,

                }
        msgJ = json.dumps(msg, indent=4, sort_keys=True, default=str)
        producer.send(myTopic,msgJ)
        

                            
                
                    
                


#Set consumer key, consumer secret, access token and access token secret
#Read them from your Twitter dev page
consumer_key = "eROgi5E9cCHSWO8A2dUZw5tD4"
consumer_secret = "RIUGtAICRdGiUrmLVsbFkbe36oROJd5NFVVHGhiGPba5bYGhEQ"
access_token = "1204688941594611712-OVpY1GbNv5Ue8Tg56L4hfWYVzTCVBT"
access_token_secret = "27dhSY2EpDZs1Bqp66Kp7o2E4awgMHdIC9y4jrId7ebU5"


# Authenticate with the OAuth protocol
# See: https://en.wikipedia.org/wiki/OAuth
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Get the API object passing the authentication object
# See http://docs.tweepy.org/en/latest/api.html to discover the available methods
api = tweepy.API(auth)

producer = KafkaProducer(
    
    bootstrap_servers=["kafka:9092"],
    value_serializer=lambda v: json.dumps(v).encode("utf-8"))


listener = MyListener(producer=producer, producer_topic="tweets")
# We need an api to stream, so we reuse the api object we create before.
# Thus, we create a Stream object.
stream = Stream(auth = api.auth, tweet_mode='extended', listener=listener)

# A number of twitter streams are available through Tweepy.
# For more information on the capabilities and limitations of the different
# streams see Twitter Streaming API Documentation.
hashtag_list =["vaccine,vaccines,Pfizer,AstraZeneca,Jhonson&Jhonson,Moderna,Novavax,Vaccinate,covidvaccine"]

# In this example we will use filter to stream all tweets containing the word "Milano".
# The track parameter is an array of search terms to stream.
stream.filter(languages=["en"],track=hashtag_list)